In [ ]:
import pandas as pd
from pandas import DataFrame
import math
import os
from tensorflow import keras
from keras import layers, models, optimizers
import numpy as np
from sklearn.utils import shuffle
import dataset
import events
import main
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, BatchNormalization, Dropout, Normalization, Activation, Input


SEGMENT_SIZE = 40
SRC_DIR = 'dataset/bob_minutes_20m_parsed_v1'

In [8]:
x, y = dataset.universal_dir_to_split(SRC_DIR, SEGMENT_SIZE)
# yy = y #tf.squeeze(y, axis=1)
x.shape, y.shape

Processing:  dataset/alright_train_20m/76_2020-06-28 104940.csv
Margin:  2


KeyError: 'swarming'

In [ ]:
train_ration = .75
X_train, X_test = x[:int(len(x) * train_ration)], x[int(len(x) * train_ration):]
Y_train, Y_test = y[:int(len(y) * train_ration)], y[int(len(y) * train_ration):]
print(f'Shapes: X_train: {X_train.shape}, Y_train: {Y_train.shape}, X_test: {X_test.shape}, Y_test: {Y_test.shape}')
X_train.shape[1], X_train.shape[2]

Shapes: X_train: (4210, 40, 1), Y_train: (4210, 1, 6), X_test: (1404, 40, 1), Y_test: (1404, 1, 6)


(40, 1)

In [ ]:
queencell_output_y = Y_train[:, 0, 0]
swarming_output_y = Y_train[:, 0, 1]
honey_output_y = Y_train[:, 0, 2]
feeding_output_y = Y_train[:, 0, 3]
treatment_output_y = Y_train[:, 0, 4]
died_output_y = Y_train[:, 0, 5]

test_queencell_output_y = Y_test[:, 0, 0]
test_swarming_output_y = Y_test[:, 0, 1]
test_honey_output_y = Y_test[:, 0, 2]
test_feeding_output_y = Y_test[:, 0, 3]
test_treatment_output_y = Y_test[:, 0, 4]
test_died_output_y = Y_test[:, 0, 5]

train_dataset = tf.data.Dataset.from_tensor_slices(({ "input": X_train }, { 
    'queencell_output': queencell_output_y,
    'swarming_output': swarming_output_y,
    'honey_output': honey_output_y,
    'feeding_output': feeding_output_y,
    'treatment_output': treatment_output_y,
    'died_output': died_output_y, 
}))
test_dataset = tf.data.Dataset.from_tensor_slices(({ "input": X_test }, { 
    'queencell_output': test_queencell_output_y,
    'swarming_output': test_swarming_output_y,
    'honey_output': test_honey_output_y, 
    'feeding_output': test_feeding_output_y, 
    'treatment_output': test_treatment_output_y, 
    'died_output': test_died_output_y, 
}))

train_dataset = train_dataset.shuffle(buffer_size=100).batch(32)
test_dataset = test_dataset.batch(32)

2024-05-29 23:55:57.305373: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Max
2024-05-29 23:55:57.305401: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2024-05-29 23:55:57.305409: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2024-05-29 23:55:57.305447: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-05-29 23:55:57.305464: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [ ]:
print(f'Train Queencell output %: {(np.sum(queencell_output_y) / len(queencell_output_y)) * 100}')
print(f'Train Swarming output %: {(np.sum(swarming_output_y) / len(swarming_output_y)) * 100}')
print(f'Train Honey output %: {(np.sum(honey_output_y) / len(honey_output_y)) * 100}')
print(f'Train Feeding output %: {(np.sum(feeding_output_y) / len(feeding_output_y)) * 100}')
print(f'Train Treatment output %: {(np.sum(treatment_output_y) / len(treatment_output_y)) * 100}')
print(f'Train Died output %: {(np.sum(died_output_y) / len(died_output_y)) * 100}')
print('\n======\n')
print(f'Test Queencell output %: {(np.sum(test_queencell_output_y) / len(test_queencell_output_y)) * 100}')
print(f'Test Swarming output %: {(np.sum(test_swarming_output_y) / len(test_swarming_output_y)) * 100}')
print(f'Test Honey output %: {(np.sum(test_honey_output_y) / len(test_honey_output_y)) * 100}')
print(f'Test Feeding output %: {(np.sum(test_feeding_output_y) / len(test_feeding_output_y)) * 100}')
print(f'Test Treatment output %: {(np.sum(test_treatment_output_y) / len(test_treatment_output_y)) * 100}')
print(f'Test Died output %: {(np.sum(test_died_output_y) / len(test_died_output_y)) * 100}')

Train Queencell output %: 2.8266033254156766
Train Swarming output %: 3.2304038004750595
Train Honey output %: 7.315914489311163
Train Feeding output %: 12.850356294536816
Train Treatment output %: 2.8266033254156766
Train Died output %: 0.40380047505938244


Test Queencell output %: 7.621082621082621
Test Swarming output %: 0.0
Test Honey output %: 3.632478632478633
Test Feeding output %: 9.971509971509972
Test Treatment output %: 8.475783475783476
Test Died output %: 0.0


In [ ]:
def build_branch(input_shape, name):
    tmp_layer = layers.LSTM(SEGMENT_SIZE*5, activation="relu")(input_shape)
    tmp_layer = layers.Dense(SEGMENT_SIZE*5, activation='relu')(tmp_layer)
    tmp_layer= Dense(1)(tmp_layer)
    tmp_layer= Activation("sigmoid", name=name)(tmp_layer)
    return tmp_layer

inputs = Input(shape=(X_train.shape[1], X_train.shape[2]), name="input")

queencell_inputs = build_branch(inputs, 'queencell_output')
swarming_inputs = build_branch(inputs, 'swarming_output')
honey_inputs = build_branch(inputs, 'honey_output')
feeding_inputs = build_branch(inputs, 'feeding_output')
treatment_inputs = build_branch(inputs, 'treatment_output')
died_inputs = build_branch(inputs, 'died_output')

losses = {
    "queencell_output": tf.losses.BinaryCrossentropy(),
    "swarming_output": tf.losses.BinaryCrossentropy(),
    "honey_output": tf.losses.BinaryCrossentropy(),
    "feeding_output": tf.losses.BinaryCrossentropy(),
    "treatment_output": tf.losses.BinaryCrossentropy(),
    "died_output": tf.losses.BinaryCrossentropy()
}
loss_weights = {
    "queencell_output": 1.0,
    "swarming_output": 1.0,
    "honey_output": 1.0,
    "feeding_output": 1.0,
    "treatment_output": 1.0,
    "died_output": 1.0
}

model = Model(
        inputs=inputs,
        outputs=[queencell_inputs, swarming_inputs, honey_inputs, feeding_inputs, treatment_inputs, died_inputs],
        name="model")
model.compile(
    optimizer='adam',
    loss=losses,
    metrics=["accuracy"],
    loss_weights=loss_weights,
)

In [ ]:
model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=32,
    batch_size=128
)

Epoch 1/32


2024-05-29 23:55:59.658815: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


132/132 [==============================] - 734s 6s/step - loss: 970305896448.0000 - queencell_output_loss: 295754.5938 - swarming_output_loss: 1.2816 - honey_output_loss: 42408.5273 - feeding_output_loss: 0.4162 - treatment_output_loss: 970305634304.0000 - died_output_loss: 0.1554 - queencell_output_accuracy: 0.9684 - swarming_output_accuracy: 0.9601 - honey_output_accuracy: 0.9230 - feeding_output_accuracy: 0.8715 - treatment_output_accuracy: 0.9682 - died_output_accuracy: 0.9895 - val_loss: 572664.3125 - val_queencell_output_loss: 0.3231 - val_swarming_output_loss: 0.0000e+00 - val_honey_output_loss: 0.2614 - val_feeding_output_loss: 0.3247 - val_treatment_output_loss: 572663.5000 - val_died_output_loss: 0.0000e+00 - val_queencell_output_accuracy: 0.9238 - val_swarming_output_accuracy: 1.0000 - val_honey_output_accuracy: 0.9637 - val_feeding_output_accuracy: 0.9003 - val_treatment_output_accuracy: 0.9152 - val_died_output_accuracy: 1.0000
Epoch 2/32
 59/132 [============>............

KeyboardInterrupt: 